In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from maml.apps.bowsr.model.megnet import MEGNet
from maml.apps.bowsr.optimizer import BayesianOptimizer
from pymatgen.core.periodic_table import get_el_sp
model = MEGNet()

def expected_radius(struct):
    element_list = struct.composition.chemical_system.split('-')
    element_list = [get_el_sp(e) for e in element_list]
    ele1, ele2 = sorted(element_list, key=lambda x: x.atomic_radius)[:2]
    return ele1.atomic_radius + ele2.atomic_radius

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-10-02 21:40:48.219960: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


# Load cubic and orthorhombic perovskite

In [2]:
from pymatgen.core.structure import Structure

cubic = Structure.from_file("cubic_perovskite.cif")
orthorhombic = Structure.from_file("orthorhombic_perovskite.cif")

# Distorted structures

In [3]:
compressed_cubic = cubic.copy()
compressed_cubic.scale_lattice(cubic.volume * 0.8)
expanded_cubic = cubic.copy()
expanded_cubic.scale_lattice(cubic.volume * 1.2)

compressed_orthorhombic = orthorhombic.copy()
compressed_orthorhombic.scale_lattice(orthorhombic.volume * 0.8)
expanded_orthorhombic = orthorhombic.copy()
expanded_orthorhombic.scale_lattice(orthorhombic.volume * 1.2)

# Experiment on cubic perovskite

In [4]:
ef_cubic = model.predict_energy(cubic)
print("MEGNet predicted formation energy of cubic perovskite: {} eV/atom".format(ef_cubic))
ef_compressed_cubic = model.predict_energy(compressed_cubic)
print("MEGNet predicted formation energy of compressed cubic perovskite: {} eV/atom".format(ef_compressed_cubic))
ef_expanded_cubic = model.predict_energy(expanded_cubic)
print("MEGNet predicted formation energy of expanded cubic perovskite: {} eV/atom".format(ef_expanded_cubic))

2021-10-02 21:41:00.264260: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


MEGNet predicted formation energy of cubic perovskite: -3.5446841716766357 eV/atom
MEGNet predicted formation energy of compressed cubic perovskite: -3.091189384460449 eV/atom
MEGNet predicted formation energy of expanded cubic perovskite: -3.3141798973083496 eV/atom


In [5]:
compressed_optimizer = BayesianOptimizer(
    model=model, 
    structure=compressed_cubic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
compressed_optimizer.set_bounds()
compressed_optimizer.optimize(n_init=100, n_iter=100, alpha=0.026 ** 2)

In [6]:
radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_compressed_cubic, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
print("MEGNet predicted formation energy of relaxed compressed cubic: {} eV/atom".format(model.predict_energy(relaxed_from_compressed_cubic)))

MEGNet predicted formation energy of relaxed compressed cubic: -3.557279109954834 eV/atom


In [7]:
expanded_optimizer = BayesianOptimizer(
    model=model, 
    structure=expanded_cubic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
expanded_optimizer.set_bounds()
expanded_optimizer.optimize(n_init=100, n_iter=100, alpha=0.026 ** 2)

In [8]:
radius = max(round(expected_radius(expanded_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_expanded_cubic, _ = expanded_optimizer.get_optimized_structure_and_energy(radius=radius)
print("MEGNet predicted formation energy of relaxed expanded cubic: {} eV/atom".format(
    model.predict_energy(relaxed_from_expanded_cubic)))

MEGNet predicted formation energy of relaxed expanded cubic: -3.5568583011627197 eV/atom


# Experiment on orthorhombic perovskite

In [30]:
ef_orthorhombic = model.predict_energy(orthorhombic)
print("MEGNet predicted formation energy of orthorhombic perovskite: {} eV/atom".format(ef_orthorhombic))
ef_compressed_orthorhombic = model.predict_energy(compressed_orthorhombic)
print("MEGNet predicted formation energy of compressed orthorhombic perovskite: {} eV/atom".format(ef_compressed_orthorhombic))
ef_expanded_orthorhombic = model.predict_energy(expanded_orthorhombic)
print("MEGNet predicted formation energy of expanded orthorhombic perovskite: {} eV/atom".format(ef_expanded_orthorhombic))

MEGNet predicted formation energy of orthorhombic perovskite: -3.5569562911987305 eV/atom
MEGNet predicted formation energy of compressed orthorhombic perovskite: -3.0739383697509766 eV/atom
MEGNet predicted formation energy of expanded orthorhombic perovskite: -3.261291027069092 eV/atom


In [36]:
compressed_optimizer = BayesianOptimizer(
    model=model, 
    structure=compressed_orthorhombic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
compressed_optimizer.set_bounds()
compressed_optimizer.optimize(n_init=100, n_iter=100, alpha=0.026 ** 2)

In [37]:
radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_compressed_orthorhombic, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
print("MEGNet predicted formation energy of relaxed compressed orthorhombic: {} eV/atom".format(
    model.predict_energy(relaxed_from_compressed_orthorhombic)))

MEGNet predicted formation energy of relaxed compressed orthorhombic: -3.562537670135498 eV/atom


In [40]:
expanded_optimizer = BayesianOptimizer(
    model=model, 
    structure=expanded_orthorhombic, 
    relax_coords=True, 
    relax_lattice=True,
    use_symmetry=True, 
    seed=42
)
expanded_optimizer.set_bounds()
expanded_optimizer.optimize(n_init=100, n_iter=100, alpha=0.026 ** 2)

In [41]:
radius = max(round(expected_radius(expanded_optimizer.structure) * 0.6, 2), 1.1)
relaxed_from_expanded_orthorhombic, _ = expanded_optimizer.get_optimized_structure_and_energy(radius=radius)
print("MEGNet predicted formation energy of relaxed expanded orthorhombic: {} eV/atom".format(
    model.predict_energy(relaxed_from_expanded_orthorhombic)))

MEGNet predicted formation energy of relaxed expanded orthorhombic: -3.5610156059265137 eV/atom
